# National Health and Nutrition Examination Survey data

### 1. data setting

In [104]:
# load modules
# basic
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

## graph
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## statistics
from scipy.stats import pearsonr
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import bartlett
from scipy.stats import levene, ttest_ind
from scipy.stats import chi2_contingency

In [2]:
# load data
hepatitis = pd.read_csv('../data/hepatitis.csv')

### 2. Checking Data

In [3]:
# 데이터 타입 확인
hepatitis.dtypes

SEQN          int64
DMDHRAGE      int64
DMDHRGND      int64
ALQ101      float64
ALQ110      float64
ALQ120Q     float64
ALQ120U     float64
ALQ130      float64
ALQ141Q     float64
ALQ141U     float64
ALQ151      float64
ALQ160      float64
LBXSAL      float64
LBXPLTSI    float64
LBXSCH      float64
LBXSTB      float64
LBXTR       float64
LBXSAPSI    float64
LBXSASSI    float64
LBXSATSI    float64
LBXHBS      float64
LBDHBG      float64
LBXHBC      float64
LBXSCR      float64
LBXIN       float64
LBXSCU      float64
LBXHCG      float64
LBXHCR      float64
DR1TALCO    float64
DR2TALCO    float64
HEQ010      float64
HEQ020      float64
HEQ030      float64
HEQ040      float64
IMQ020        int64
LBXGLU      float64
MCQ170L     float64
MCQ180L     float64
MCQ240M     float64
MCQ160L     float64
dtype: object

In [4]:
# 담당변수
my_value = ['SEQN', 'LBXHBC', 'LBXHCG',
'LBXHCR', 'LBXGLU', 'LBXSCR', 'LBXIN', 'IMQ020', 'HEQ010',
'HEQ020', 'HEQ030', 'HEQ040', 'MCQ170L', 'MCQ180L']

In [5]:
# 해당 변수 확인
m = hepatitis[my_value]
m.head(10)

,SEQN,LBXHBC,LBXHCG,LBXHCR,LBXGLU,LBXSCR,LBXIN,IMQ020,HEQ010,HEQ020,HEQ030,HEQ040,MCQ170L,MCQ180L
0,73557,1.0,NaN,NaN,NaN,1.21,NaN,3,2.0,NaN,2.0,NaN,NaN,NaN
1,73558,2.0,NaN,NaN,NaN,0.79,NaN,3,2.0,NaN,2.0,NaN,NaN,NaN
2,73559,2.0,NaN,NaN,193.0,1.22,5.83,3,2.0,NaN,2.0,NaN,NaN,NaN
3,73560,2.0,NaN,NaN,NaN,NaN,NaN,1,2.0,NaN,2.0,NaN,NaN,NaN
4,73561,2.0,NaN,NaN,107.0,0.73,6.12,3,2.0,NaN,2.0,NaN,NaN,NaN
5,73562,2.0,NaN,NaN,NaN,0.89,NaN,9,2.0,NaN,2.0,NaN,NaN,NaN
6,73563,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
7,73564,2.0,NaN,NaN,110.0,0.92,14.91,3,2.0,NaN,2.0,NaN,NaN,NaN
8,73565,NaN,NaN,NaN,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,NaN,NaN
9,73566,2.0,NaN,NaN,NaN,0.55,NaN,3,2.0,NaN,2.0,NaN,NaN,NaN


In [6]:
# 결측 확인
na_percent = round(m.isna().sum()/len(m)*100,2)
print(na_percent)

# 결측치 확인 그래프 퍼센트 구하기
n_p = pd.DataFrame(na_percent).reset_index()
n_p.columns = ['index', 'p']

SEQN        0.00
LBXHBC     24.76
LBXHCG     99.34
LBXHCR     98.65
LBXGLU     68.83
LBXSCR     35.60
LBXIN      69.60
IMQ020      0.00
HEQ010     15.75
HEQ020     99.35
HEQ030     15.75
HEQ040     99.25
MCQ170L    97.70
MCQ180L    97.70
dtype: float64


In [7]:
# top 10 graphs
fig = go.Figure()

fig.add_trace(go.Bar(x = n_p['index'],
                     y = n_p['p']
                     ))

fig.update_layout(title_text="데이터 결측치 퍼센트",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')

fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

In [8]:
# describe
m.describe()

,SEQN,LBXHBC,LBXHCG,LBXHCR,LBXGLU,LBXSCR,LBXIN,IMQ020,HEQ010,HEQ020,HEQ030,HEQ040,MCQ170L,MCQ180L
count,10175.000000,7656.000000,67.000000,137.000000,3172.000000,6553.000000,3093.000000,10175.000000,8572.000000,66.000000,8572.000000,76.000000,234.000000,234.000000
mean,78644.000000,1.942921,1.746269,1.510949,105.084489,0.880172,13.526809,2.501032,2.014349,3.242424,2.013999,2.144737,2.004274,1751.183761
std,2937.413829,0.232009,1.520997,0.501715,32.060429,0.487262,18.638388,2.299557,0.402309,2.893285,0.410787,1.888237,2.135934,12984.320749
min,73557.000000,1.000000,1.000000,1.000000,51.000000,0.290000,0.140000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
25%,76100.500000,2.000000,1.000000,1.000000,92.000000,0.690000,6.080000,1.000000,2.000000,2.000000,2.000000,1.000000,1.000000,32.000000
50%,78644.000000,2.000000,1.000000,2.000000,98.000000,0.820000,9.470000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,42.500000
75%,81187.500000,2.000000,2.000000,2.000000,107.000000,0.980000,15.350000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,54.000000
max,83731.000000,2.000000,9.000000,2.000000,421.000000,17.410000,682.480000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,99999.000000


### 3. Make a Graph

In [9]:
# histogram of glucose
fig = px.histogram(m, x="LBXGLU", marginal='box')
fig.update_layout(title_text="글루코스",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 정상범위 대략 70 ~ 99로 이루어져 있으며 최저 51에서 최고 421로 이루어져 있음
- 당뇨 환자는 300까지 허용될 수 있기 때문에 추가적으로 보는 수치는 300 이상인 값을 확인해봐야 알 수 있을것 같음
- 당뇨에 관한 질병 요인이 끼어 있을 수 있기 때문에 조절이 필요할 수도 있어 보임

In [10]:
# histogram of creatinine
fig = px.histogram(m, x="LBXSCR", marginal='box')
fig.update_layout(title_text="크레아티닌",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 연령, 체중에 따라 다른 지표를 사용하지만 통상적으로 0.4 ~ 1.5정도를 정상 범위로 확인하는 것 같음
- 최대 5를 넘지 않는다고 하여 5 이상인 경우를 확인해 볼 예정
- 10을 넘으면 재확인이 필요하다는 견해
- 2이상인 사람들은 신장병이나 투석환자일 가능성이 매우 높음

In [11]:
# histogram of insulin
fig = px.histogram(m, x="LBXIN", marginal='box')
fig.update_layout(title_text="인슐린",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

- 최대 허용 수치는 30 이는 식후 나올 수 있는 정상 수치로 판단할 수 있음
- 당뇨환자 아니면 보통 검사를 안하기 때문에 인슐린 검사를 진행한 환자에 대해 신장 질환을 가지고 있을 가능성이 있음
- 결측치를 가지고 있는 사람들은 검사 안함 과 같은 영역으로 표시해야 할것 같음
- 식후 1시간 뒤에 나오는 인슐린 값은 70까지도 나오는 것 같음
- 70보다 큰 값을 다시한번 확인해볼 필요가 있음

### 4. Categorical values

In [12]:
# HEQ line
m['HEQ010'].value_counts()

HEQ010
2.0    8479
1.0      66
9.0      27
Name: count, dtype: int64

In [13]:
m['HEQ020'].value_counts()

HEQ020
2.0    44
9.0    13
1.0     9
Name: count, dtype: int64

In [14]:
m['HEQ030'].value_counts()

HEQ030
2.0    8468
1.0      76
9.0      28
Name: count, dtype: int64

In [15]:
m['HEQ040'].value_counts()

HEQ040
2.0    47
1.0    24
9.0     5
Name: count, dtype: int64

In [16]:
m['MCQ170L'].value_counts()

MCQ170L
1.0    132
2.0     83
9.0     19
Name: count, dtype: int64

In [17]:
# 연속형 변수에 방해되는 값 치우기
m['MCQ180L'][m['MCQ180L'] == 99999] = np.nan

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\3430064038.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
# histogram of insulin
fig = px.histogram(m, x="MCQ180L", marginal='box')
fig.update_layout(title_text="간 질환 진단 나이",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

# 백신 접종 여부와 나이 비교해볼 것.

In [19]:
# histogram of insulin
fig = px.histogram(m, x="MCQ180L", color="IMQ020" , marginal='box')
fig.update_layout(title_text="간 질환 진단 나이",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff')
fig.show()

In [20]:
m['MCQ180L'][m['IMQ020'] == 3].value_counts()

MCQ180L
50.0    10
60.0     7
30.0     7
45.0     5
40.0     5
57.0     5
54.0     4
33.0     4
65.0     4
35.0     4
42.0     4
37.0     4
32.0     4
67.0     3
63.0     3
55.0     3
52.0     3
25.0     3
41.0     3
46.0     2
68.0     2
53.0     2
62.0     2
36.0     2
27.0     2
47.0     2
38.0     2
43.0     2
48.0     2
69.0     2
28.0     1
21.0     1
19.0     1
12.0     1
5.0      1
13.0     1
10.0     1
71.0     1
70.0     1
17.0     1
44.0     1
34.0     1
29.0     1
23.0     1
20.0     1
59.0     1
15.0     1
56.0     1
73.0     1
51.0     1
72.0     1
49.0     1
64.0     1
26.0     1
Name: count, dtype: int64

- HEQ010, HEQ030은 결측치인 이유가 검사를 받지 않아서 DK인 9번으로 채우는 것이 합리적이여 보임
- HEQ020, HEQ040은 값을 가지고 있는 경우가 앞선 변수 HEQ010, HEQ030에서 Yes라고 답한 경우만 존재함
- 따라서 새로운 칼럼을 만들어서 10: Not diagnosed 값을 넣고 분석하는 방법이 괜찮아 보임
- MCQ170L -> MCQ180L 또한 연계질문
- 180L은 진단 당시 연령이기 때문에 다른 값으로 채우기 불가능 -> np.nan

In [21]:
# 값 채우기
m['HEQ010'][m['HEQ010'].isna()] = 9
m['HEQ030'][m['HEQ030'].isna()] = 9
m['HEQ020'][m['HEQ020'].isna()] = 10
m['HEQ040'][m['HEQ040'].isna()] = 10
m['MCQ170L'][m['MCQ170L'].isna()] = 9


C:\Users\abc\AppData\Local\Temp\ipykernel_21272\2058428072.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\2058428072.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\2058428072.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\2058428072.py:5: SettingWithCopyWarning:


A value is tryi

In [22]:
# 백신 접종 여부
m['IMQ020'].value_counts()

IMQ020
1    5230
3    3517
9     968
2     453
7       7
Name: count, dtype: int64

In [23]:
# 7을 9로 합체
m['IMQ020'][m['IMQ020'] == 7] = 9

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\3016181658.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 5. Chect the outlier and imputate median

In [24]:
# 글루코스 이상치
m['LBXGLU'][m['LBXGLU'] > 300] = m['LBXGLU'].median()
m['LBXGLU'][m['LBXGLU'].isna()] = m['LBXGLU'].median()

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1082779378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1082779378.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
# 크레아티닌 이상치
m['LBXSCR'][m['LBXSCR'] > 5] = m['LBXSCR'].median()
m['LBXSCR'][m['LBXSCR'].isna()] = m['LBXSCR'].median()

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\3081696749.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\3081696749.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
# 인슐린 이상치
m['LBXIN'][m['LBXIN'] > 70] = m['LBXIN'].median()
m['LBXIN'][m['LBXIN'].isna()] = m['LBXIN'].median()

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\4129458282.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\4129458282.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 6. Imputate Outlier

In [27]:
# 결측치 확인
m.isna().sum()

SEQN           0
LBXHBC      2519
LBXHCG     10108
LBXHCR     10038
LBXGLU         0
LBXSCR         0
LBXIN          0
IMQ020         0
HEQ010         0
HEQ020         0
HEQ030         0
HEQ040         0
MCQ170L        0
MCQ180L     9945
dtype: int64

In [28]:
# 사용할 데이터
use_data = m[m.columns.difference(['LBXHBC', 'LBXHCG', 'LBXHCR', 'MCQ180L', 
                                   'HEQ030', 'HEQ040'])]
use_data

,HEQ010,HEQ020,IMQ020,LBXGLU,LBXIN,LBXSCR,MCQ170L,SEQN
0,2.0,10.0,3,98.0,9.47,1.21,9.0,73557
1,2.0,10.0,3,98.0,9.47,0.79,9.0,73558
2,2.0,10.0,3,193.0,5.83,1.22,9.0,73559
3,2.0,10.0,1,98.0,9.47,0.82,9.0,73560
4,2.0,10.0,3,107.0,6.12,0.73,9.0,73561
...,...,...,...,...,...,...,...,...
10170,2.0,10.0,3,108.0,3.76,1.10,9.0,83727
10171,9.0,10.0,2,98.0,9.47,0.82,9.0,83728
10172,2.0,10.0,1,98.0,9.47,0.82,9.0,83729
10173,2.0,10.0,1,98.0,9.47,0.82,9.0,83730


In [29]:
# load data
hb = pd.read_csv('../data/hepatitis_bottom.csv')
hm1 = pd.read_csv('../data/filtered_hep_도연.csv')
ht = pd.read_csv('../data/filtered_hep_동혁.csv')

In [30]:
# extract
use_hb = hb[['MCQ160L', 'DR1TALCO',
             'DR2TALCO', 'ALQ101', 'ALQ151', 'SEQN']]
use_ht = ht.iloc[:, 1:-1]

In [31]:
# change the attributes
use_hb[['DR1TALCO', 'DR2TALCO']] = use_hb[['DR1TALCO', 'DR2TALCO']].astype(float)

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\2572417649.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
# merge data
use_data2 = pd.merge(use_hb, hm1, on = 'SEQN', how = 'inner')
use_data3 = pd.merge(use_data2, use_ht, on = 'SEQN', how = 'inner')
use_data4 = pd.merge(use_data3, use_data, on = 'SEQN', how = 'inner')

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1976734943.py:3: UserWarning:

You are merging on int and float columns where the float values are not equal to their int representation.



In [33]:
ht

,Unnamed: 0,SEQN,DMDHRAGE,DMDHRGND,LBXSAL,LBXPLTSI,LBXSCH,LBXSTB,LBXTR,Current_HBV
0,0,73557.0,69.0,1.0,4.1,204.0,168.0,0.8,88.0,2.0
1,1,73558.0,54.0,1.0,4.7,314.0,167.0,0.9,88.0,2.0
2,2,73559.0,72.0,1.0,3.7,237.0,127.0,0.6,51.0,2.0
3,3,73560.0,33.0,1.0,4.3,240.0,181.0,0.6,88.0,2.0
4,4,73561.0,78.0,1.0,4.3,300.0,207.0,0.5,75.0,2.0
...,...,...,...,...,...,...,...,...,...,...
7645,10167,83724.0,80.0,1.0,3.8,180.0,155.0,0.5,88.0,2.0
7646,10168,83725.0,25.0,1.0,4.3,270.0,181.0,0.6,88.0,2.0
7647,10170,83727.0,19.0,2.0,4.9,226.0,195.0,0.9,81.0,2.0
7648,10172,83729.0,42.0,2.0,4.1,213.0,148.0,0.2,88.0,2.0


In [34]:
# check na
use_data4.isna().sum().sort_values(ascending=False)

LBXSCU         5141
DR2TALCO       1530
LBXSASSI       1105
LBXSATSI       1105
LBXSAPSI       1104
DR1TALCO        734
MCQ160L           0
LBXSTB            0
LBXSCR            0
LBXIN             0
LBXGLU            0
IMQ020            0
HEQ020            0
HEQ010            0
LBXTR             0
DMDHRGND          0
LBXSCH            0
LBXPLTSI          0
LBXSAL            0
DMDHRAGE          0
Current_HBV       0
SEQN              0
ALQ151            0
ALQ101            0
MCQ170L           0
dtype: int64

In [35]:
# imputate na to median
use_data4['LBXSCU'][use_data4['LBXSCU'].isna()] = use_data4['LBXSCU'].median()
use_data4['LBXSASSI'][use_data4['LBXSASSI'].isna()] = use_data4['LBXSASSI'].median()
use_data4['LBXSATSI'][use_data4['LBXSATSI'].isna()] = use_data4['LBXSATSI'].median()
use_data4['LBXSAPSI'][use_data4['LBXSAPSI'].isna()] = use_data4['LBXSAPSI'].median()
use_data4['DR1TALCO'][use_data4['DR1TALCO'].isna()] = use_data4['DR1TALCO'].median()
use_data4['DR2TALCO'][use_data4['DR2TALCO'].isna()] = use_data4['DR2TALCO'].median()

# 조건에 맞지 않는 값을 해당 변수의 중앙 값으로 대체
use_data4.loc[use_data4['LBXSCU'] >= 250, 'LBXSCU'] = use_data4['LBXSCU'].median()
use_data4.loc[use_data4['LBXSAPSI'] >= 600, 'LBXSAPSI'] = use_data4['LBXSAPSI'].median()
use_data4.loc[use_data4['LBXSASSI'] >= 600, 'LBXSASSI'] = use_data4['LBXSASSI'].median()
use_data4.loc[use_data4['LBXSATSI'] >= 400, 'LBXSATSI'] = use_data4['LBXSATSI'].median()

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1605123759.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1605123759.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1605123759.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1605123759.py:5: SettingWithCopyWarning:


A value is tryi

In [36]:
# check na
use_data4.isna().sum().sort_values(ascending=False)

MCQ160L        0
LBXSAL         0
LBXSCR         0
LBXIN          0
LBXGLU         0
IMQ020         0
HEQ020         0
HEQ010         0
LBXTR          0
LBXSTB         0
LBXSCH         0
LBXPLTSI       0
DMDHRGND       0
DR1TALCO       0
DMDHRAGE       0
LBXSCU         0
LBXSATSI       0
LBXSASSI       0
LBXSAPSI       0
Current_HBV    0
SEQN           0
ALQ151         0
ALQ101         0
DR2TALCO       0
MCQ170L        0
dtype: int64

In [37]:
# save the data
use_data4.to_csv('../data/hepatitis_merge.csv', index = False)

In [38]:
# 시퀸스 넘버 제거
ud_x = use_data4[use_data4.columns.difference(['SEQN', 'Current_HBV'])]
ud_y = use_data4['Current_HBV']
ud_y[ud_y == 2] = 0

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1918315134.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
ud_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7633 entries, 0 to 7632
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ALQ101    7633 non-null   float64
 1   ALQ151    7633 non-null   float64
 2   DMDHRAGE  7633 non-null   float64
 3   DMDHRGND  7633 non-null   float64
 4   DR1TALCO  7633 non-null   float64
 5   DR2TALCO  7633 non-null   float64
 6   HEQ010    7633 non-null   float64
 7   HEQ020    7633 non-null   float64
 8   IMQ020    7633 non-null   int64  
 9   LBXGLU    7633 non-null   float64
 10  LBXIN     7633 non-null   float64
 11  LBXPLTSI  7633 non-null   float64
 12  LBXSAL    7633 non-null   float64
 13  LBXSAPSI  7633 non-null   float64
 14  LBXSASSI  7633 non-null   float64
 15  LBXSATSI  7633 non-null   float64
 16  LBXSCH    7633 non-null   float64
 17  LBXSCR    7633 non-null   float64
 18  LBXSCU    7633 non-null   float64
 19  LBXSTB    7633 non-null   float64
 20  LBXTR     7633 non-null   floa

In [40]:
# names
ud_x.columns

Index(['ALQ101', 'ALQ151', 'DMDHRAGE', 'DMDHRGND', 'DR1TALCO', 'DR2TALCO',
       'HEQ010', 'HEQ020', 'IMQ020', 'LBXGLU', 'LBXIN', 'LBXPLTSI', 'LBXSAL',
       'LBXSAPSI', 'LBXSASSI', 'LBXSATSI', 'LBXSCH', 'LBXSCR', 'LBXSCU',
       'LBXSTB', 'LBXTR', 'MCQ160L', 'MCQ170L'],
      dtype='object')

### 7. OneHotEncoding

In [41]:
# No is Default, Gender : Women
ud_x[['ALQ101', 'ALQ151', 'DMDHRGND',
'HEQ010', 'HEQ020',
'MCQ160L', 'MCQ170L']] = ud_x[['ALQ101', 'ALQ151', 'DMDHRGND',
'HEQ010', 'HEQ020',
'MCQ160L', 'MCQ170L']].replace(2.0, 0.0)

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\3609031972.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
# no vaccine is default
ud_x['IMQ020'][ud_x['IMQ020'] == 3.0] = 0.0

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\120009257.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
# change datatype
ud_x[['ALQ101', 'ALQ151', 'DMDHRGND', 'HEQ010', 'HEQ020',
'MCQ160L', 'MCQ170L', 'IMQ020']] = ud_x[['ALQ101', 'ALQ151', 'DMDHRGND',
'HEQ010', 'HEQ020','MCQ160L', 'MCQ170L', 'IMQ020']].astype(int)

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1778071767.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [44]:
# get dummies
ud_x = pd.get_dummies(ud_x, columns = ['ALQ101', 'ALQ151',
'HEQ010', 'HEQ020', 'MCQ160L', 'MCQ170L', 'IMQ020'], drop_first=True)

In [45]:
# boolin to numeric values
ud_x = ud_x.replace(True, 1).replace(False, 0)

### 8. Logistic Regression

In [46]:
# logistic regression
model = sm.Logit(ud_y, ud_x)
results = model.fit()

         Current function value: 0.024520
         Iterations: 35


c:\Users\abc\anaconda3\envs\ss\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [47]:
# 분석 결과
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Current_HBV   No. Observations:                 7633
Model:                          Logit   Df Residuals:                     7601
Method:                           MLE   Df Model:                           31
Date:                Tue, 14 Nov 2023   Pseudo R-squ.:                  0.2182
Time:                        20:25:46   Log-Likelihood:                -187.16
converged:                      False   LL-Null:                       -239.41
Covariance Type:            nonrobust   LLR p-value:                 6.829e-10
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
DMDHRAGE      -0.0065      0.012     -0.554      0.580      -0.029       0.016
DMDHRGND      -0.1755      0.356     -0.492      0.623      -0.874       0.523
DR1TALCO      -0.0075      0.010     -0.780      0.436      -0.026       0.011
DR2TALCO      -0.0456      0.036     -1.255      0.210      -0.117       0.026
LBXGLU        -0.0339      0.021     -1.606      0.108      -0.075       0.007
LBXIN          0.0113      0.028      0.404      0.686      -0.043       0.066
LBXPLTSI      -0.0089      0.004     -2.477      0.013      -0.016      -0.002
LBXSAL         0.0153      0.543      0.028      0.978      -1.050       1.080
LBXSAPSI       0.0045      0.005      0.971      0.331      -0.005       0.014
LBXSASSI       0.0009      0.008      0.109      0.913      -0.015       0.016
LBXSATSI       0.0140      0.006      2.451      0.014       0.003       0.025
LBXSCH         0.0021      0.004      0.478      0.633      -0.006       0.011
LBXSCR         0.4649      0.454      1.024      0.306      -0.425       1.355
LBXSCU         0.0086      0.011      0.768      0.443      -0.013       0.031
LBXSTB         0.5908      0.551      1.073      0.283      -0.488       1.670
LBXTR         -0.0020      0.003     -0.579      0.563      -0.009       0.005
ALQ101_1       0.2711      0.655      0.414      0.679      -1.012       1.554
ALQ101_9      -0.5858      0.621     -0.943      0.346      -1.803       0.632
ALQ151_1       0.3814      0.592      0.644      0.519      -0.779       1.542
ALQ151_9       1.0005      0.680      1.471      0.141      -0.333       2.334
HEQ010_1      27.6228   1.23e+06   2.25e-05      1.000    -2.4e+06     2.4e+06
HEQ010_9     -16.3576   1.01e+04     -0.002      0.999   -1.98e+04    1.98e+04
HEQ020_1      -0.4681      1.256     -0.373      0.709      -2.930       1.994
HEQ020_9     -34.0323    1.6e+07  -2.13e-06      1.000   -3.14e+07    3.14e+07
HEQ020_10     23.3490   1.23e+06    1.9e-05      1.000    -2.4e+06     2.4e+06
MCQ160L_1    -27.1312   1.23e+06  -2.21e-05      1.000    -2.4e+06     2.4e+06
MCQ160L_9     -1.3054      0.810     -1.612      0.107      -2.893       0.282
MCQ170L_1      1.0665      1.369      0.779      0.436      -1.616       3.749
MCQ170L_9    -26.2726   1.23e+06  -2.14e-05      1.000    -2.4e+06     2.4e+06
IMQ020_1      -0.1177      0.424     -0.277      0.781      -0.950       0.714
IMQ020_2       0.9342      0.857      1.090      0.276      -0.746       2.614
IMQ020_9       0.3126      0.536      0.583      0.560      -0.738       1.363
==============================================================================
"""

In [48]:
# 계수와 신뢰구간 추출
coef = results.params
conf_int = results.conf_int()
conf_int['diff'] = conf_int.iloc[:,1] - conf_int.iloc[:, 0]
names = ud_x.columns
color = ['lightslategray' if conf_int.iloc[i,0]*conf_int.iloc[i,1] < 0 else 'red' for i in range(len(conf_int))]

In [49]:
# 변수별 최소값, 최대값을 하나의 막대 그래프로 표현
fig = go.Figure()

for i in range(len(coef)):
    variable_name = names[i]
    
    range_color = color[i]
    range_height = conf_int.iloc[i, 1] - conf_int.iloc[i, 0]

    # 하나의 막대 그래프로 최소값과 최대값을 표현
    fig.add_trace(go.Bar(
        x=[variable_name],
        y=[range_height],
        base=[conf_int.iloc[i, 0]],
        marker=dict(color=range_color, line=dict(width=0.5)),
        showlegend=False,
        name=variable_name,
    ))

# y=0 직선 추가
fig.add_trace(go.Scatter(
    x=names,
    y=[0] * len(names),
    mode='lines',
    line=dict(color='green', width=2),
    name='y=0 line'
))

# 그래프 레이아웃 설정
fig.update_layout(
    title_text="각 변수의 신뢰구간 형태",
    title_x=0.5,
    title_xanchor='center',
    title_font_size=25,
    title_font_color='black',
    title_font_family='NanumSquare',
    plot_bgcolor='#ffffff',
    xaxis=dict(title='신뢰구간'),
    yaxis=dict(title='변수', range = [-0.3, 0.3]),
    showlegend=False,
    bargap = 0
)

fig.update_traces(
    marker_opacity=0.4,
)

fig.show()

In [50]:
# 변수별 최소값, 최대값을 하나의 막대 그래프로 표현
fig = go.Figure()

for i in range(len(coef)):
    variable_name = names[i]
    
    range_color = color[i]
    range_height = conf_int.iloc[i, 1] - conf_int.iloc[i, 0]

    # 하나의 막대 그래프로 최소값과 최대값을 표현
    fig.add_trace(go.Bar(
        x=[variable_name],
        y=[range_height],
        base=[conf_int.iloc[i, 0]],
        marker=dict(color=range_color, line=dict(width=0.5)),
        showlegend=False,
        name=variable_name,
    ))

# y=0 직선 추가
fig.add_trace(go.Scatter(
    x=names,
    y=[0] * len(names),
    mode='lines',
    line=dict(color='green', width=2),
    name='y=0 line'
))

# 그래프 레이아웃 설정
fig.update_layout(
    title_text="각 변수의 신뢰구간 형태",
    title_x=0.5,
    title_xanchor='center',
    title_font_size=25,
    title_font_color='black',
    title_font_family='NanumSquare',
    plot_bgcolor='#ffffff',
    xaxis=dict(title='신뢰구간'),
    yaxis=dict(title='변수'),
    showlegend=False,
    bargap = 0
)

fig.update_traces(
    marker_opacity=0.4,
)

fig.show()

In [57]:
# out of categorical variables
ud_x2 = ud_x[['DMDHRAGE', 'DMDHRGND', 'DR1TALCO', 'DR2TALCO', 'LBXGLU', 'LBXIN',
       'LBXPLTSI', 'LBXSAL', 'LBXSAPSI', 'LBXSASSI', 'LBXSATSI', 'LBXSCH', 
       'LBXSCR', 'LBXSCU', 'LBXSTB', 'LBXTR', 'IMQ020_1', 'IMQ020_2', 'IMQ020_9']]

In [58]:
# logistic regression
model = sm.Logit(ud_y, ud_x2)
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.028761
         Iterations 11


In [59]:
# 분석 결과
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            Current_HBV   No. Observations:                 7633
Model:                          Logit   Df Residuals:                     7614
Method:                           MLE   Df Model:                           18
Date:                Tue, 14 Nov 2023   Pseudo R-squ.:                 0.08302
Time:                        20:27:38   Log-Likelihood:                -219.53
converged:                       True   LL-Null:                       -239.41
Covariance Type:            nonrobust   LLR p-value:                  0.002255
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
DMDHRAGE    5.162e-05      0.011      0.005      0.996      -0.022       0.022
DMDHRGND      -0.2437      0.334     -0.730      0.465      -0.898       0.410
DR1TALCO      -0.0062      0.011     -0.555      0.579      -0.028       0.016
DR2TALCO      -0.0391      0.035     -1.111      0.267      -0.108       0.030
LBXGLU        -0.0328      0.017     -1.907      0.056      -0.067       0.001
LBXIN          0.0114      0.025      0.451      0.652      -0.038       0.061
LBXPLTSI      -0.0111      0.003     -3.276      0.001      -0.018      -0.004
LBXSAL        -0.2129      0.385     -0.553      0.580      -0.967       0.541
LBXSAPSI      -0.0004      0.004     -0.085      0.932      -0.009       0.008
LBXSASSI   -5.062e-06      0.008     -0.001      0.999      -0.015       0.015
LBXSATSI       0.0144      0.005      2.645      0.008       0.004       0.025
LBXSCH         0.0042      0.004      1.065      0.287      -0.004       0.012
LBXSCR         0.2068      0.497      0.416      0.677      -0.766       1.180
LBXSCU        -0.0020      0.010     -0.194      0.846      -0.022       0.018
LBXSTB         0.6556      0.502      1.306      0.192      -0.328       1.640
LBXTR          0.0003      0.003      0.104      0.917      -0.005       0.006
IMQ020_1      -0.4356      0.401     -1.086      0.278      -1.222       0.351
IMQ020_2       0.6953      0.812      0.856      0.392      -0.897       2.287
IMQ020_9       0.0205      0.517      0.040      0.968      -0.993       1.034
==============================================================================
"""

In [60]:
# 계수와 신뢰구간 추출
coef = results.params
conf_int = results.conf_int()
conf_int['diff'] = conf_int.iloc[:,1] - conf_int.iloc[:, 0]
names = ud_x2.columns
color = ['lightslategray' if conf_int.iloc[i,0]*conf_int.iloc[i,1] < 0 else 'red' for i in range(len(conf_int))]

In [61]:
# 변수별 최소값, 최대값을 하나의 막대 그래프로 표현
fig = go.Figure()

for i in range(len(coef)):
    variable_name = names[i]
    
    range_color = color[i]
    range_height = conf_int.iloc[i, 1] - conf_int.iloc[i, 0]

    # 하나의 막대 그래프로 최소값과 최대값을 표현
    fig.add_trace(go.Bar(
        x=[variable_name],
        y=[range_height],
        base=[conf_int.iloc[i, 0]],
        marker=dict(color=range_color, line=dict(width=0.5)),
        showlegend=False,
        name=variable_name,
    ))

# y=0 직선 추가
fig.add_trace(go.Scatter(
    x=names,
    y=[0] * len(names),
    mode='lines',
    line=dict(color='green', width=2),
    name='y=0 line'
))

# 그래프 레이아웃 설정
fig.update_layout(
    title_text="각 변수의 신뢰구간 형태",
    title_x=0.5,
    title_xanchor='center',
    title_font_size=25,
    title_font_color='black',
    title_font_family='NanumSquare',
    plot_bgcolor='#ffffff',
    xaxis=dict(title='신뢰구간'),
    yaxis=dict(title='변수', range = [-0.3, 0.3]),
    showlegend=False,
    bargap = 0
)

fig.update_traces(
    marker_opacity=0.4,
)

fig.show()

In [62]:
# 변수별 최소값, 최대값을 하나의 막대 그래프로 표현
fig = go.Figure()

for i in range(len(coef)):
    variable_name = names[i]
    
    range_color = color[i]
    range_height = conf_int.iloc[i, 1] - conf_int.iloc[i, 0]

    # 하나의 막대 그래프로 최소값과 최대값을 표현
    fig.add_trace(go.Bar(
        x=[variable_name],
        y=[range_height],
        base=[conf_int.iloc[i, 0]],
        marker=dict(color=range_color, line=dict(width=0.5)),
        showlegend=False,
        name=variable_name,
    ))

# y=0 직선 추가
fig.add_trace(go.Scatter(
    x=names,
    y=[0] * len(names),
    mode='lines',
    line=dict(color='green', width=2),
    name='y=0 line'
))

# 그래프 레이아웃 설정
fig.update_layout(
    title_text="각 변수의 신뢰구간 형태",
    title_x=0.5,
    title_xanchor='center',
    title_font_size=25,
    title_font_color='black',
    title_font_family='NanumSquare',
    plot_bgcolor='#ffffff',
    xaxis=dict(title='신뢰구간'),
    yaxis=dict(title='변수'),
    showlegend=False,
    bargap = 0
)

fig.update_traces(
    marker_opacity=0.4,
)

fig.show()

### 9. demographic data

In [102]:
hepatitis['DMDHRGND'].value_counts()

DMDHRGND
1    5088
2    5087
Name: count, dtype: int64

In [71]:
# select variables
chosen = hm1[['SEQN', 'Current_HBV']]

# merge data
d = pd.merge(chosen, hepatitis, on = 'SEQN', how = 'left')

In [74]:
data = d[['Current_HBV', 'DMDHRAGE', 'DMDHRGND', 'ALQ101',
       'ALQ151', 'LBXSAL', 'LBXPLTSI', 'LBXSCH', 'LBXSTB',
       'LBXTR', 'LBXSAPSI',
       'LBXSASSI', 'LBXSATSI', 'LBXSCR', 'LBXIN',
       'LBXSCU', 'DR1TALCO', 'DR2TALCO', 'HEQ010',
       'HEQ020', 'IMQ020', 'LBXGLU', 'MCQ170L',
       'MCQ160L']]

In [85]:
# object data changes
data[['Current_HBV', 'DMDHRGND', 'ALQ101', 'ALQ151', 'HEQ010',
       'HEQ020', 'IMQ020', 'MCQ170L', 'MCQ160L']] = data[['Current_HBV', 'DMDHRGND', 'ALQ101', 'ALQ151', 'HEQ010',
       'HEQ020', 'IMQ020', 'MCQ170L', 'MCQ160L']].astype(str)

C:\Users\abc\AppData\Local\Temp\ipykernel_21272\1632298838.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [94]:
# 연속형 변수만 가져오기
data_continuous = data[list(data.dtypes[data.dtypes != 'object'].index)]
data_continuous.columns

Index(['DMDHRAGE', 'LBXSAL', 'LBXPLTSI', 'LBXSCH', 'LBXSTB', 'LBXTR',
       'LBXSAPSI', 'LBXSASSI', 'LBXSATSI', 'LBXSCR', 'LBXIN', 'LBXSCU',
       'DR1TALCO', 'DR2TALCO', 'LBXGLU'],
      dtype='object')

In [95]:
# 연속형 변수에서의 99999와 77777은 전부 NA
data_continuous = data_continuous.replace(99999, np.nan).replace(77777, np.nan)

In [96]:
# 범주형 변수만 가져오기
data_categorical = data[list(data.dtypes[data.dtypes == 'object'].index)]
data_categorical.columns

Index(['Current_HBV', 'DMDHRGND', 'ALQ101', 'ALQ151', 'HEQ010', 'HEQ020',
       'IMQ020', 'MCQ170L', 'MCQ160L'],
      dtype='object')

In [124]:
# 기준점인 stage 추가
data_continuous['Current_HBV'] = data['Current_HBV']
data_continuous.columns

Index(['DMDHRAGE', 'LBXSAL', 'LBXPLTSI', 'LBXSCH', 'LBXSTB', 'LBXTR',
       'LBXSAPSI', 'LBXSASSI', 'LBXSATSI', 'LBXSCR', 'LBXIN', 'LBXSCU',
       'DR1TALCO', 'DR2TALCO', 'LBXGLU', 'Current_HBV'],
      dtype='object')

In [125]:
# 연속형 변수 중앙값(표준오차)
data_continuous.groupby('Current_HBV').median()

,DMDHRAGE,LBXSAL,LBXPLTSI,LBXSCH,LBXSTB,LBXTR,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSCR,LBXIN,LBXSCU,DR1TALCO,DR2TALCO,LBXGLU
Current_HBV,,,,,,,,,,,,,,,
1.0,50.0,4.3,203.5,182.0,0.7,86.5,65.0,29.0,28.5,0.82,10.735,110.0,0.0,0.0,98.0
2.0,46.0,4.3,238.0,180.0,0.6,88.0,66.0,22.0,20.0,0.82,9.450,115.1,0.0,0.0,98.0


In [126]:
round(data_continuous.groupby('Current_HBV').std(), 2)

,DMDHRAGE,LBXSAL,LBXPLTSI,LBXSCH,LBXSTB,LBXTR,LBXSAPSI,LBXSASSI,LBXSATSI,LBXSCR,LBXIN,LBXSCU,DR1TALCO,DR2TALCO,LBXGLU
Current_HBV,,,,,,,,,,,,,,,
1.0,14.85,0.35,58.38,42.32,0.33,89.99,39.21,15.33,28.91,0.38,7.53,28.5,9.01,4.77,11.89
2.0,15.68,0.34,60.97,41.97,0.31,115.60,51.70,17.75,18.38,0.49,18.68,29.2,24.21,16.76,32.01


In [132]:
# 범주변수 count
for i in data_categorical.columns:
    if i != 'Current_HBV':
        print(f'{i} 변수 결과')
        print(pd.crosstab(data[i], data['Current_HBV']))

DMDHRGND 변수 결과
Current_HBV  1.0   2.0
DMDHRGND              
1             19  3943
2             19  3669
ALQ101 변수 결과
Current_HBV  1.0   2.0
ALQ101                
1.0           19  3639
2.0           13  1538
9.0            1     7
nan            5  2428
ALQ151 변수 결과
Current_HBV  1.0   2.0
ALQ151                
1.0            4   699
2.0           18  3595
9.0            0     1
nan           16  3317
HEQ010 변수 결과
Current_HBV  1.0   2.0
HEQ010                
1.0           10    52
2.0           28  7539
9.0            0    21
HEQ020 변수 결과
Current_HBV  1.0   2.0
HEQ020                
1.0            1     8
2.0            9    33
9.0            0    11
nan           28  7560
IMQ020 변수 결과
Current_HBV  1.0   2.0
IMQ020                
1             12  3574
2              2   142
3             19  3089
7              1     6
9              4   801
MCQ170L 변수 결과
Current_HBV  1.0   2.0
MCQ170L               
1.0            3   121
2.0            1    72
9.0            0    16
nan      

In [128]:
for i in data_categorical.columns:
    if i != 'Current_HBV':
        print(f'{i} 변수 결과')
        tmp = pd.crosstab(data['Current_HBV'], data[i])
        chi2, p, dof, expected = chi2_contingency(tmp)
        msg = 'Test Statistic: {}\np-value: {}\nDegree of Freedom: {}'
        print(msg.format(round(chi2, 2), round(p, 4), dof))
        print(expected)

DMDHRGND 변수 결과
Test Statistic: 0.0
p-value: 0.9531
Degree of Freedom: 1
[[  19.68052288   18.31947712]
 [3942.31947712 3669.68052288]]
ALQ101 변수 결과
Test Statistic: 31.19
p-value: 0.0
Degree of Freedom: 3
[[1.81704575e+01 7.70431373e+00 3.97385621e-02 1.20854902e+01]
 [3.63982954e+03 1.54329569e+03 7.96026144e+00 2.42091451e+03]]
ALQ151 변수 결과
Test Statistic: 0.1
p-value: 0.9921
Degree of Freedom: 3
[[3.49202614e+00 1.79469281e+01 4.96732026e-03 1.65560784e+01]
 [6.99507974e+02 3.59505307e+03 9.95032680e-01 3.31644392e+03]]
HEQ010 변수 결과
Test Statistic: 309.1
p-value: 0.0
Degree of Freedom: 2
[[3.07973856e-01 3.75877124e+01 1.04313725e-01]
 [6.16920261e+01 7.52941229e+03 2.08956863e+01]]
HEQ020 변수 결과
Test Statistic: 395.38
p-value: 0.0
Degree of Freedom: 3
[[4.47058824e-02 2.08627451e-01 5.46405229e-02 3.76920261e+01]
 [8.95529412e+00 4.17913725e+01 1.09453595e+01 7.55030797e+03]]
IMQ020 변수 결과
Test Statistic: 31.98
p-value: 0.0
Degree of Freedom: 4
[[1.78128105e+01 7.15294118e-01 1.543843

In [129]:
# 연속형 변수
for i in data_continuous.columns:
    if i != 'Current_HBV':
        # extract data with not null values
        data2 = data_continuous[~data_continuous[i].isna()]

        # Extract data for different 'Status' categories
        lv_1 = data2[i][(data2['Current_HBV'] == '1.0')]
        lv_2 = data2[i][(data2['Current_HBV'] == '2.0')]


        # Perform ANOVA
        t_statistic, p_value = ttest_ind(lv_1, lv_2)
        print(f'value : {i}')
        print(f'Indep t-statistic: {t_statistic:.2f}')
        print(f'Indep p-value: {p_value:.4f}')
        print()

value : DMDHRAGE
Indep t-statistic: 0.66
Indep p-value: 0.5087

value : LBXSAL
Indep t-statistic: -0.15
Indep p-value: 0.8784

value : LBXPLTSI
Indep t-statistic: -3.79
Indep p-value: 0.0002

value : LBXSCH
Indep t-statistic: 0.79
Indep p-value: 0.4324

value : LBXSTB
Indep t-statistic: 1.69
Indep p-value: 0.0915

value : LBXTR
Indep t-statistic: 0.02
Indep p-value: 0.9866

value : LBXSAPSI
Indep t-statistic: -0.53
Indep p-value: 0.5940

value : LBXSASSI
Indep t-statistic: 3.08
Indep p-value: 0.0021

value : LBXSATSI
Indep t-statistic: 5.11
Indep p-value: 0.0000

value : LBXSCR
Indep t-statistic: 0.36
Indep p-value: 0.7153

value : LBXIN
Indep t-statistic: -0.37
Indep p-value: 0.7119

value : LBXSCU
Indep t-statistic: -0.51
Indep p-value: 0.6098

value : DR1TALCO
Indep t-statistic: -0.55
Indep p-value: 0.5790

value : DR2TALCO
Indep t-statistic: -0.89
Indep p-value: 0.3758

value : LBXGLU
Indep t-statistic: -1.08
Indep p-value: 0.2808

